In [1]:
##################################################################################
#
#       Make Heat Index
#       By Cascade Tuholske Spring 2021
#
#       Program makes daily maximum heat index tifs with CHIRTS-daily Tmax and 
#       relative humidity min estimated with CHIRTS-daily Tmax
#
#       NOAA Heat Index Equation - https://www.wpc.ncep.noaa.gov/html/heatindex_equation.shtml
#
#       Note - right now units are all in C. They can be updated to F or C as needed. 
#              See make_hi function to make changes CPT July 2021
#
#################################################################################

#### Dependencies
import numpy as np
import pandas as pd
import xarray 
import os
import glob
import rasterio
import matplotlib.pyplot as plt
import time
import multiprocessing as mp 
from multiprocessing import Pool
import multiprocessing
import ClimFuncs

In [2]:
def parallel_loop(function, start_list, cpu_num):
    """Run a routine in parallel
    Args: 
        function = function to apply in parallel
        start_list = list of args to loop through in parallel
        cpu_num = numper of cpus to fire  
    """ 
    start = time.time()
    pool = Pool(processes = cpu_num)
    pool.map(function, start_list)
    # pool.map_async(function, dir_list)
    pool.close()

    end = time.time()
    print(end-start)

In [3]:
def hi_loop(year):
    
    """
    """
    print(multiprocessing.current_process(), year)
    
    # Set up file paths
    rh_path = os.path.join('/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum-Tmax/', str(year)) # RH min made with CHIRTS-daily Tmax
    tmax_path = os.path.join('/home/chc-data-out/products/CHIRTSdaily/v1.0/global_tifs_p05/Tmax/', str(year)) # CHIRTS-daily Tmax
    
    # zip RH and Tmax
    rh_fns = sorted(glob.glob(rh_path+'/*.tif'))
    tmax_fns = sorted(glob.glob(tmax_path+'/*.tif'))
    zipped_list = list(zip(rh_fns,tmax_fns))
     
    test = zipped_list[0:3]
    
    for i, fns in enumerate(test):
        
        with rasterio.open(fns[0]) as rh:
            rh_arr = rh.read(1)
            meta = rh.meta
        
        with rasterio.open(fns[1]) as tmax:
            tmax_arr = tmax.read(1)
            meta = rh.meta
        
        arr = tmax_arr * rh_arr
        
        fn_out = '/scratch/cascade/UEH-daily/himax/' + str(year) + '/test' + str(i) + '.tif'
        
        with rasterio.open(fn_out, 'w', **meta) as out:
            out.write_band(1, arr)

In [4]:
# Make years 
year_list = list(range(1983,2016+1))
year_sub = year_list[0:4]

In [5]:
# Run it
parallel_loop(function = hi_loop, start_list = year_sub, cpu_num = 4)

<ForkProcess(ForkPoolWorker-1, started daemon)> 1983
<ForkProcess(ForkPoolWorker-4, started daemon)> 1986
<ForkProcess(ForkPoolWorker-2, started daemon)> 1984
<ForkProcess(ForkPoolWorker-3, started daemon)> 1985
1.05949068069458
